# Создание базы данных

In [1]:
import sqlite3

In [21]:
# create_queries = ['''
# CREATE TABLE "texts" ( 
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 
# '''
# CREATE TABLE "sents" (
# 	"id"	INTEGER,
# 	"text_id"	INTEGER,
# 	"sent"	TEXT,
# 	"lemmatized"	TEXT,
# 	"pos_in_text"	INTEGER,
# 	FOREIGN KEY("text_id") REFERENCES "texts"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 

# '''
# CREATE TABLE "govern_models" (
# 	"id"	INTEGER,
# 	"model"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_imgs" (
# 	"id"	INTEGER,
# 	"math_tagname"	TEXT,
# 	"filename"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_tags" (
# 	"id"	INTEGER,
# 	"feature"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "lemmas" (
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "pos" (
# 	"id"	INTEGER,
# 	"name"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "deprels" (
# 	"id"	INTEGER,
# 	"name"	INTEGER,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "tokens" (
# 	"id"	INTEGER,
# 	"sent_id"	INTEGER,
# 	"word_in_sent"	INTEGER,
# 	"token"	TEXT,
# 	FOREIGN KEY("sent_id") REFERENCES "sents"("id") ON DELETE RESTRICT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''', 

# '''
# CREATE TABLE "math_annotation" (
# 	"id"	INTEGER,
# 	"id_token_start"	INTEGER,
# 	"id_token_end"	INTEGER,
# 	"feature_id"	INTEGER,
# 	"govern_model_id"	INTEGER,
# 	FOREIGN KEY("feature_id") REFERENCES "math_tags"("id"),
# 	FOREIGN KEY("govern_model_id") REFERENCES "govern_models"("id"),
# 	FOREIGN KEY("id_token_start") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("id_token_end") REFERENCES "tokens"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "grammar_annotation" (
# 	"token_id"	INTEGER,
# 	"lemma_id"	INTEGER,
# 	"pos_id"	INTEGER,
# 	"deprel_id"	INTEGER,
# 	"head_id"	INTEGER,
# 	FOREIGN KEY("lemma_id") REFERENCES "lemmas"("id"),
# 	FOREIGN KEY("token_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("deprel_id") REFERENCES "deprels"("id"),
# 	FOREIGN KEY("head_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("pos_id") REFERENCES "pos"("id"),
# 	PRIMARY KEY("token_id")
# );
# '''
#                  ]

In [23]:
# db_path = 'math_corpus_database.db'
# conn = sqlite3.connect(db_path, check_same_thread=False)
# cur = conn.cursor()

In [103]:
# for q in create_queries:
#     cur.execute(q)
#     conn.commit()
# conn.close()

# Парсинг и заполнение БД

In [2]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 15.3/15.3 MB 992.0 kB/s eta 0:00:00


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import spacy
nlp = spacy.load("ru_core_news_sm")

[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [5]:
db_path = 'math_corpus_database.db'

In [92]:
class DBHandler:
    conn = None
    cur = None

    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cur = self.conn.cursor()

    def __del__(self):
        self.conn.close()

    def add_text(self, text_name):
        
        self.cur.execute('''INSERT or IGNORE INTO texts (name)
                            VALUES (?)
                            RETURNING id''', (text_name,))
    
        added_id = self.cur.fetchone()
        self.conn.commit()
        
        if not added_id:
            self.cur.execute('''SELECT id
                                FROM texts
                                WHERE name = (?)''', (text_name, ))
            added_id = self.cur.fetchone()
        return added_id[0]
        
    def add_sent(self, text_id, sent, lemmatized, sent_count):
        
        self.cur.execute('''INSERT INTO sents (text_id, sent, lemmatized, pos_in_text)
                            VALUES (?, ?, ?, ?)
                            RETURNING id''', (text_id, sent, lemmatized, sent_count))
        
        added_id = self.cur.fetchone()
        self.conn.commit()
        return added_id[0]
    
    def add_tokens(self, tokens_info): 
        ":param tokens_info: list of tuples (sent_id, word_in_sent, token, char_start, char_end)"
        
        self.cur.executemany('''INSERT INTO tokens (sent_id, word_in_sent, token, char_start, char_end)
                                VALUES (?, ?, ?, ?, ?)''', tokens_info)
        
        self.conn.commit()
        
        self.cur.execute('''SELECT id 
                            FROM tokens
                            WHERE sent_id = (?)
                            ORDER BY word_in_sent''', (tokens_info[0][0],))
        added_ids = self.cur.fetchall()
        return [token_id[0] for token_id in added_ids]
    
    
    def add_lemmas(self, lemmas):
        lemmas = [(el,) for el in lemmas]
        self.cur.executemany('''INSERT or IGNORE
                            INTO lemmas (name)
                            VALUES (?)''', lemmas)
        self.conn.commit()
        
        lemma_ids = []
        for lemma in lemmas:
            self.cur.execute('''SELECT id
                                FROM lemmas
                                WHERE name = ?''', lemma)
            lemma_ids.append(self.cur.fetchone()[0])
        return lemma_ids
    
    
    def add_grammar_annot(self, token_grammar_info):
        ":param token_grammar_info: [(token_id, pos, deprel, head_id, lemma)]"
        
        self.cur.executemany('''INSERT INTO grammar_annotation (token_id, lemma_id, pos_id, deprel_id, head_id)
                                SELECT ?, 
                                        lemmas.id, 
                                        (SELECT pos.id FROM pos WHERE pos.name = ?),
                                        (SELECT deprels.id FROM deprels WHERE deprels.name = ?),
                                        ?
                                FROM lemmas
                                WHERE lemmas.name = ?''', token_grammar_info)
        self.conn.commit()
        
    def add_math_tags(self, tags):
        tags = [(el,) for el in tags]
        self.cur.executemany('''INSERT or IGNORE
                                INTO math_tags (feature)
                                VALUES (?)''', tags)
        self.conn.commit()
        
    def add_govern_models(self, models):
        models = [(el,) for el in models]
        self.cur.executemany('''INSERT or IGNORE
                                INTO govern_models (model)
                                VALUES (?)''', models)
        self.conn.commit()
        
    def get_sent_id(self, sentence):
        self.cur.execute('''SELECT id
                            FROM sents
                            WHERE sents.sent = (?)''', (sentence,))
        return self.cur.fetchone()[0]
    
    
    def add_math_annotation(self, annotation):
        """:param annotation: [{
            'sentence': str, 
            'segment_text': str,
            'char_start': int, 
            'char_end': int,
            'annot': str, 
            'govern_model': str}
            }]"""
        
        annot = list(set([an['annot'] for an in annotation]))
        self.add_math_tags(annot)
        govern_models = list(set([an['govern_model'] for an in annotation if an['govern_model']]))
        self.add_govern_models(govern_models)
        
        sent_ids = {}
        for annot in annotation:
            if not sent_ids.get(annot['sentence']):
                sent_ids[annot['sentence']] = self.get_sent_id(annot['sentence'])
            annot['sentence_id'] = sent_ids[annot['sentence']]
            
        self.cur.executemany('''INSERT INTO math_annotation (feature_id, govern_model_id, id_token_start, id_token_end)
                                SELECT math_tags.id, 
                                (SELECT govern_models.id FROM govern_models WHERE govern_models.model = :govern_model),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent 
                                LIMIT 1),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent DESC
                                LIMIT 1)
                            FROM math_tags
                            WHERE math_tags.feature = :annot''', annotation)
        self.conn.commit()
        
        
db = DBHandler(db_path)

In [93]:
xml_filename = 'test.xml'

In [94]:
from bs4 import BeautifulSoup

In [95]:
with open(xml_filename, 'r', encoding='utf-8') as f:
    xml_file = f.read()
bs_data = BeautifulSoup(xml_file, "xml")
segs = bs_data.find_all('segment')

In [96]:
def parse_sentence(analysed_sent):
    parsed = {
        'tokens': [], 
        'lemmas': [], 
        'poses': [], 
        'deprels': [], 
        'head_id_in_sent': [], 
        'full_sentence': analysed_sent.text,
    }
    first_word_in_sent = 0
    
    for t in analysed_sent:
        parsed['tokens'].append(t.text)
        parsed['lemmas'].append(t.lemma_)
        parsed['poses'].append(t.pos_)
        parsed['deprels'].append(t.dep_.lower())
        
        if t.is_sent_start:
            first_word_in_sent = t.i
        
        parsed['head_id_in_sent'].append(t.head.i - first_word_in_sent)
    parsed['lemmatized_sent'] = ' '.join(parsed['lemmas'])
    return parsed

In [97]:
import re

In [98]:
def accum_token_info(parsed, sent_id):
    n_tokens = len(parsed['tokens'])
    token_info = []
    sent_string = parsed['full_sentence']
    prev_chars_count = 0
    for i in range(n_tokens):
        
        token = parsed['tokens'][i]
        m = re.search(re.escape(token), sent_string)
        char_start, char_end = m.start(), m.end()
        
        token_info.append((sent_id, i, parsed['tokens'][i], prev_chars_count + char_start, prev_chars_count + char_end))
        prev_chars_count += char_end
        sent_string = sent_string[char_end:]  # обрезаем строку, чтобы короткие слова не искались в начале строки
    return token_info

In [99]:
def accum_grammar_info(parsed, added_tokens):
    n_tokens = len(parsed['tokens'])
    head_ids = [added_tokens[hi] for hi in parsed['head_id_in_sent']]
    token_grammar_info = [
    (
        added_tokens[i], 
        parsed['poses'][i],
        parsed['deprels'][i], 
        head_ids[i],
        parsed['lemmas'][i]
    ) for i in range(n_tokens)
    ]
    return token_grammar_info

In [100]:
text_id = 1
sent_count = 0
annotation = []
segment_info= {}
for seg in segs:
    parent_id = seg.get('parent')
    if not parent_id:
        text = seg.get_text()
        if segment_info:
            annotation.append(segment_info)
        segment_info = {
            'text': text, 
            'segments_annot': []
        }
        sentences = list(nlp(text).sents)
        for sent in sentences:
            sent_count += 1
            parsed_sentence = parse_sentence(sent)
            
            lemmas = parsed_sentence['lemmas']
            added_lemmas = db.add_lemmas(lemmas)
            
            sent_text = parsed_sentence['full_sentence']
            sent_lemmatized = parsed_sentence['lemmatized_sent']
            sent_id = db.add_sent(text_id, sent_text, sent_lemmatized, sent_count)
            
            tokens_info = accum_token_info(parsed_sentence, sent_id)
            added_tokens = db.add_tokens(tokens_info)
    
            
            grammar_info = accum_grammar_info(parsed_sentence, added_tokens)
            db.add_grammar_annot(grammar_info)
    else:
        if int(parent_id) == int(seg.get('id')) - 1:
            continue
        else:
            segment_text = seg.get_text()
            segment_info['segments_annot'].append((segment_text, seg.get('features'), seg.get('comment')))

In [101]:
# annotation

In [102]:
def find_segment(annotation_piece):
    text, segments = annotation_piece['text'], annotation_piece['segments_annot']
    
    sentences = list(nlp(text).sents)
    sentences_starts = [s.start_char for s in sentences]
    prev_chars_count = 0
    segments_info = []
    for seg in segments:
        m = re.search('(?<![A-я])' + re.escape(seg[0]) + '(?![A-я])', text)  # чтобы не находило в середине слова
        local_start, local_end = m.start(), m.end()  # начало и конец совпадения в обрезанной строке
        text = text[local_end:]
        global_start, global_end = local_start + prev_chars_count, local_end + prev_chars_count
        segment_sent = sentences_starts.index([s for s in sentences_starts if s <= global_start][-1])
        prev_chars_count = global_end
        segments_info.append({'sentence': sentences[segment_sent].text, 
                              'segment_text': seg[0],
                              'char_start': global_start - sentences_starts[segment_sent], 
                              'char_end': global_end - sentences_starts[segment_sent],
                              'annot': seg[1], 
                            'govern_model': seg[2]})
    return segments_info

In [103]:
all_math_annotation = []
for annot in annotation:
    all_math_annotation.extend(find_segment(annot))

In [104]:
# all_math_annotation

In [105]:
db.add_math_annotation(all_math_annotation)

In [13]:
segs

[<segment features="math_tags" filename="" id="1" state="active" subcorp=""><segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment> с <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment> <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>.</segment>,
 <segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment>,
 <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment>,
 <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>,
 <segment feat

In [133]:
t = 'Показатель делимого, из показателя делимого вычитают показатель делителя.Например: два в пятой степени разделим на два в третьей степени, вычитаем показатели, получаем два во второй степени и четыре.'